In [1]:
# import packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import csv
from sklearn.impute import KNNImputer

In [9]:
# load data: is "5_has_wettercode" the latest version?

df = pd.read_csv("/workspaces/bakery_sales_prediction/sourcedata/cleaned_data/6.4_temperature_deviation.csv")

print(df.head(), "\n")

df_wetter: 
         Datum  Bewoelkung  Temperatur  Windgeschwindigkeit  Wettercode  \
0  2013-07-01         6.0     17.8375                 15.0        20.0   
1  2013-07-01         6.0     17.8375                 15.0        20.0   
2  2013-07-01         6.0     17.8375                 15.0        20.0   
3  2013-07-01         6.0     17.8375                 15.0        20.0   
4  2013-07-01         6.0     17.8375                 15.0        20.0   

   KielerWoche  Warengruppe      Umsatz  Feiertage  Ferientage  
0          0.0          1.0  148.828353        0.0         1.0  
1          0.0          2.0  535.856285        0.0         1.0  
2          0.0          3.0  201.198426        0.0         1.0  
3          0.0          4.0   65.890169        0.0         1.0  
4          0.0          5.0  317.475875        0.0         1.0   

df_no_wetter: 
         Datum  Bewoelkung  Temperatur  Windgeschwindigkeit  Wettercode  \
0  2013-07-02         3.0     17.3125                 10.0  

## 3. Wochentage

In [17]:
df_w = df.copy()

In [18]:
# extract date features
df_w['year'] = df_w['Datum'].dt.year
df_w['month'] = df_w['Datum'].dt.month
df_w['week'] = df_w['Datum'].dt.isocalendar().week
df_w['weekday'] = df_w['Datum'].dt.dayofweek
df_w['day_month'] = df_w['Datum'].dt.day
print(df_w.head())


       Datum  Bewoelkung  Temperatur  Windgeschwindigkeit  Wettercode  \
0 2013-07-01         6.0     17.8375                 15.0        20.0   
1 2013-07-01         6.0     17.8375                 15.0        20.0   
2 2013-07-01         6.0     17.8375                 15.0        20.0   
3 2013-07-01         6.0     17.8375                 15.0        20.0   
4 2013-07-01         6.0     17.8375                 15.0        20.0   

   KielerWoche  Warengruppe      Umsatz  Feiertage  Ferientage  Wetterklasse  \
0          0.0          1.0  148.828353        0.0         1.0             6   
1          0.0          2.0  535.856285        0.0         1.0             6   
2          0.0          3.0  201.198426        0.0         1.0             6   
3          0.0          4.0   65.890169        0.0         1.0             6   
4          0.0          5.0  317.475875        0.0         1.0             6   

   Niederschlag  year  month  week  weekday  day_month  
0           0.3  2013  

In [19]:
print(df_w["month"].describe())

count    6999.000000
mean        6.581226
std         3.670393
min         1.000000
25%         3.000000
50%         7.000000
75%        10.000000
max        12.000000
Name: month, dtype: float64


In [20]:
# define the season
def get_season(month):
	    if month in [12, 1, 2]:
	        return 'winter', 4
	    elif month in [3, 4, 5]:
	        return 'spring', 1
	    elif month in [6, 7, 8]:
	        return 'summer', 2
	    else:
	        return 'autumn', 3
	
df_w['season_str'], df_w['season'] = zip(*df_w['month'].apply(get_season))


# Create binary features for seasons --> creates one column per season and then 0/1
# probably to generic for predictions?

#weather_data['winter'] = (weather_data['season'] == 'winter').astype(int)
#weather_data['spring'] = (weather_data['season'] == 'spring').astype(int)
#weather_data['summer'] = (weather_data['season'] == 'summer').astype(int)
#weather_data['autumn'] = (weather_data['season'] == 'autumn').astype(int)


print(df_w.head())


       Datum  Bewoelkung  Temperatur  Windgeschwindigkeit  Wettercode  \
0 2013-07-01         6.0     17.8375                 15.0        20.0   
1 2013-07-01         6.0     17.8375                 15.0        20.0   
2 2013-07-01         6.0     17.8375                 15.0        20.0   
3 2013-07-01         6.0     17.8375                 15.0        20.0   
4 2013-07-01         6.0     17.8375                 15.0        20.0   

   KielerWoche  Warengruppe      Umsatz  Feiertage  Ferientage  Wetterklasse  \
0          0.0          1.0  148.828353        0.0         1.0             6   
1          0.0          2.0  535.856285        0.0         1.0             6   
2          0.0          3.0  201.198426        0.0         1.0             6   
3          0.0          4.0   65.890169        0.0         1.0             6   
4          0.0          5.0  317.475875        0.0         1.0             6   

   Niederschlag  year  month  week  weekday  day_month season_str  season  
0   

In [21]:
# dataframe after the previous steps:
df_w


,Datum,Bewoelkung,Temperatur,Windgeschwindigkeit,Wettercode,KielerWoche,Warengruppe,Umsatz,Feiertage,Ferientage,Wetterklasse,Niederschlag,year,month,week,weekday,day_month,season_str,season
0,2013-07-01,6.0,17.8375,15.0,20.0,0.0,1.0,148.828353,0.0,1.0,6,0.3,2013,7,27,0,1,summer,2
1,2013-07-01,6.0,17.8375,15.0,20.0,0.0,2.0,535.856285,0.0,1.0,6,0.3,2013,7,27,0,1,summer,2
2,2013-07-01,6.0,17.8375,15.0,20.0,0.0,3.0,201.198426,0.0,1.0,6,0.3,2013,7,27,0,1,summer,2
3,2013-07-01,6.0,17.8375,15.0,20.0,0.0,4.0,65.890169,0.0,1.0,6,0.3,2013,7,27,0,1,summer,2
4,2013-07-01,6.0,17.8375,15.0,20.0,0.0,5.0,317.475875,0.0,1.0,6,0.3,2013,7,27,0,1,summer,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6994,2018-07-30,6.0,27.7875,10.0,21.0,0.0,1.0,148.541340,0.0,1.0,6,0.0,2018,7,31,0,30,summer,2
6995,2018-07-30,6.0,27.7875,10.0,21.0,0.0,2.0,616.358562,0.0,1.0,6,0.0,2018,7,31,0,30,summer,2
6996,2018-07-30,6.0,27.7875,10.0,21.0,0.0,3.0,348.770346,0.0,1.0,6,0.0,2018,7,31,0,30,summer,2
6997,2018-07-30,6.0,27.7875,10.0,21.0,0.0,4.0,71.822977,0.0,1.0,6,0.0,2018,7,31,0,30,summer,2


## Ergebnis speichern:

In [24]:
 # Save the updated data to new CSV files
df_w.to_csv('../sourcedata/cleaned_data/6.5_date_variables-2.csv', sep=',', index=False)